In [ ]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)
!pip install transformers

import torch
import sys
import transformers
sys.path.append('/gdrive/My Drive/reheaven-PyTorch-master/my_transformers')
sys.path.append('/gdrive/My Drive/reheaven-PyTorch-master')
from my_transformers import(
    ElectraConfig,
    ElectraForSequenceClassification,
    ElectraTokenizer,
)


labels = ['0', '1']

config = ElectraConfig.from_pretrained(
            "/gdrive/My Drive/reheaven-PyTorch-master/model_weight/wordpiece_base",
            num_labels=2,
            id2label={str(i): label for i, label in enumerate(labels)},
            label2id={label: i for i, label in enumerate(labels)},
            )

model = ElectraForSequenceClassification.from_pretrained(
            "/gdrive/My Drive/reheaven-PyTorch-master/model_weight/wordpiece_base",
            from_tf=True,
            config=config
            )
loaded = torch.load("/gdrive/My Drive/reheaven-PyTorch-master/model_weight/pytorch_model.bin",map_location='cpu')  #torch.load(PATH,map_location), map_location은 model을 load하는 device 종류
model.load_state_dict(loaded,strict=False)

tokenizer = ElectraTokenizer.from_pretrained("/gdrive/My Drive/reheaven-PyTorch-master/model_weight/wordpiece_base")

with open('/gdrive/My Drive/reheaven-PyTorch-master/test_output.txt','w',encoding='utf-8') as fd_write:
  with open('/gdrive/My Drive/reheaven-PyTorch-master/test_input.txt','r',encoding='utf-8') as fd_read:
    for sentence in fd_read.readlines():

      tokenized_sentence= tokenizer.tokenize(sentence)


      gen_encoded = tokenizer.encode(tokenized_sentence, return_tensors="pt") # model내 vocabulary에 있는 id로 토큰을 mapping(encoding)

      model_input = gen_encoded

      classification_result = model(model_input)

      result = torch.argmax(classification_result[0])
      if result:
        fd_write.write((str(1)+'\n'))
      else:
        fd_write.write((str(0)+'\n'))
print('\ndone')

Mounted at /gdrive
     |████████████████████████████████| 890kB 2.9MB/s 
     |████████████████████████████████| 890kB 17.9MB/s 
     |████████████████████████████████| 1.1MB 19.6MB/s 
     |████████████████████████████████| 3.0MB 34.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=14dceb76249deb7f8d2f9f4fd52ed0b171526e9ded23bcdffdd426257bda5771
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Skipping discriminator_predictions/dense/bias ['discriminator_predictions', 'dense', 'bias'] 'ElectraForSequenceClassification' object has no attribute 'discriminator_predictions'
Skipping discriminator_predictions/dense/bias/adam_m ['discriminator_predictions', 'dense', 'bias', 'adam_m'] 'ElectraForSequenceClassification' object has no attribute 'discriminator_predictions'
Skipping discriminator_predictions/dense/bias/adam_v ['discriminator_predictions', 